# Aula 05 - Lendo dados oceanográficos em diversos formatos (netCDF, OPeNDAP, ERDDAP etc) e dimensões (AKA além das tabelas)]

**Objetivos**

- Exibir dados em várias dimensões (Satélites, Modelos, etc)
- Ler de dados diversas fontes binárias (NetCDF, HDF4/5, e Protocolos online)
- Introduzir conceitos básicos de CDM (*Common Data Models*)

Relembrando *Slices*

One way to remember how slices work is to think of the indices as pointing between characters, with the left edge of the first character numbered 0. Then the right edge of the last character of a string of n characters has index n, for example:

     +---+---+---+---+---+---+
     | P | y | t | h | o | n |
     +---+---+---+---+---+---+
     0   1   2   3   4   5   6
    -6  -5  -4  -3  -2  -1

In [ ]:
t = 'Python'

In [ ]:
t[0:2]

In [ ]:
t[0:4]

In [ ]:
t[::-1]

In [ ]:
t[::2]

In [ ]:
import numpy as np

arr = np.array([[3, 6, 2, 1, 7],
                [4, 1, 3, 2, 8],
                [7, 9, 2, 1, 8],
                [8, 6, 9, 6, 7],
                [9, 1, 9, 2, 6],
                [9, 8, 1, 5, 6],
                [0, 4, 2, 0, 6],
                [0, 3, 1, 4, 7]])

<img src='./files/2dbase2.png', width="300">

<img src='./files/2dbase1.png', width="300">

In [ ]:
arr = np.array([[[3, 6, 2, 1, 7],
                 [4, 1, 3, 2, 8],
                 [7, 9, 2, 1, 8],
                 [8, 6, 9, 6, 7],
                 [9, 1, 9, 2, 6],
                 [9, 8, 1, 5, 6],
                 [0, 4, 2, 0, 6],
                 [0, 3, 1, 4, 7]],
                [[5, 5, 3, 9, 3],
                 [8, 3, 5, 1, 1],
                 [3, 4, 3, 0, 9],
                 [1, 4, 1, 0, 2],
                 [7, 1, 2, 0, 1],
                 [5, 1, 3, 7, 8],
                 [8, 0, 9, 6, 0],
                 [7, 7, 4, 4, 4]],
                [[1, 0, 8, 9, 1],
                 [7, 4, 8, 8, 2],
                 [9, 1, 8, 3, 6],
                 [5, 6, 2, 0, 1],
                 [7, 4, 2, 5, 7],
                 [9, 5, 6, 8, 6],
                 [7, 4, 4, 7, 1],
                 [8, 4, 4, 9, 1]]])

In [ ]:
arr.shape

<img src='./files/3darray.png' width="300">

<img src='./files/3dbase1.png', width="300">

<img src='./files/3dbase2.png', width="300">

<img src='./files/3dbase3.png', width="300">

<img src='./files/3dbase4.png', width="300">

<img src='./files/3dbase5.png', width="300">

#### > 3D

<img src='./files/4dbase.png', width="300">

<img src='./files/4dshape.png', width="300">

<img src='./files/5dbase.png', width="300">

<img src='./files/5shape.png', width="300">

Exemplo real

<img src='./files/netcdf-diagram.png', width="300">

In [ ]:
from netCDF4 import Dataset

nc = Dataset('./data/mdt_cnes_cls2009_global_v1.1.nc')

nc

In [ ]:
u = nc.variables['Grid_0002']
u

In [ ]:
v = nc.variables['Grid_0003']
v

In [ ]:
u, v = u[:], v[:]

In [ ]:
lon = nc.variables['NbLongitudes'][:]
lat = nc.variables['NbLatitudes'][:]

In [ ]:
import numpy as np

lon, lat = np.meshgrid(lon, lat)

In [ ]:
lon.shape, lat.shape, u.shape, v.shape

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
sub = 5
ax.quiver(lon[::sub, ::sub], lat[::sub, ::sub], u.T[::sub, ::sub], v.T[::sub, ::sub])

In [ ]:
from oceans import wrap_lon180

lon = wrap_lon180(lon)

In [ ]:
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.io import shapereader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


def make_map(projection=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize=(9, 13),
                           subplot_kw=dict(projection=projection))
    gl = ax.gridlines(draw_labels=True)
    gl.xlabels_top = gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

In [ ]:
#lon, lat, u.T, v.T

mask_x = np.logical_and(lon > -40, lon < -36)
mask_y = np.logical_and(lat > -15, lat < -12)
mask = np.logical_and(mask_x, mask_y)

In [ ]:
import cartopy.feature as cfeature
land_10m = cfeature.NaturalEarthFeature('physical', 'land', '10m',
                                        edgecolor='face',
                                        facecolor=cfeature.COLORS['land'])

fig, ax = make_map()
ax.quiver(lon[mask], lat[mask], u.T[mask], v.T[mask])
ax.add_feature(land_10m)
ax.coastlines('10m')
ax.set_extent([-40, -36, -15, -12])